## Knapsack problem

### Create knapsack problem

In [26]:
from QHyper.problems.knapsack import KnapsackProblem

knapsack = KnapsackProblem(max_weight=2, items=[(1, 2), (1, 2), (1, 1)])

In [27]:
print(f"Variables: {knapsack.variables}")
print(f"Objective function: {knapsack.objective_function}")
print("Constraints:")
for constraint in knapsack.constraints:
    print(f"    {constraint}")

Variables: (x0, x1, x2, x3, x4)
Objective function: -2*x0 - 2*x1 - x2
Constraints:
    -x3 - x4 + 1
    -x0 - x1 - x2 + x3 + 2*x4


### Use VQA to solve knapsack problem

In [28]:
from QHyper.solvers.vqa.base import VQA
from QHyper.solvers.vqa import WFEval, QAOA

In [29]:
solver_config = {
    'pqc': {
        'type': 'qaoa',
        'layers': 5,
    },
    'optimizer': {
        'type': 'scipy',
        'maxfun': 200
    },
    'eval_func': {
        'type': 'wfeval'
    }
}

params_cofing = {
    'angles': [[0.5]*5, [1]*5],
    'hyper_args': [1, 2.5, 2.5],
}

vqa = VQA(knapsack, config=solver_config)
best_result, optimizer_params, hyper_params = vqa.solve(params_cofing)

In [30]:
print(f"Best results: {best_result}")
print(f"Params used for optimizer: {optimizer_params},\nand params used for hyper optimizer: {hyper_params}")
# We haven't use hyper optimizer and qaoa only optimize angles, hence hyperparameters are the same as provided

Best results: -1.7860315362678931
Params used for optimizer: [[1.64230366 0.08750879 0.         0.12854769 2.49365561]
 [0.93907822 0.62299303 1.10213747 0.41506598 1.45096723]],
and params used for hyper optimizer: [1.  2.5 2.5]


In [31]:
params_cofing = {
    'angles': optimizer_params,
    'hyper_args': hyper_params,
}
# You can provide VQA with objects instead of cofinguration dict
# Here we create VQA instance, without optimizer, which will results geting just a quantum circuit (no optimization inside)
vqa = VQA(knapsack, QAOA(layers=5), None, WFEval())
best_result, _, _ = vqa.solve(params_cofing)
print(f"Results are the same as above: {best_result}")

Results are the same as above: -1.7860315362678931


#### Using hyper optimizers

In [32]:
from QHyper.optimizers.random import Random

solver_config = {
    'pqc': {
        'type': 'qaoa',
        'layers': 5,
    },
    'optimizer': {
        'type': 'scipy',
        'maxfun': 200
    },
    'eval_func': {
        'type': 'wfeval'
    }
}
params_cofing = {
    'angles': [[0.5]*5, [1]*5],
    'hyper_args': [1, 2.5, 2.5],
}
vqa = VQA(knapsack, config=solver_config)

random = Random(processes=5, number_of_samples=100, bounds=[[1, 10], [1, 10], [1, 10]])
best_result_h, optimizer_params_h, hyper_params_h = vqa.solve(params_cofing, random)

100%|██████████| 100/100 [05:17<00:00,  3.17s/it]


In [33]:
print(f"Best results from hyper optimizer: {best_result_h}")
print(f"Params used for optimizer: {optimizer_params_h},\nand params used for hyper optimizer: {hyper_params_h}")

Best results from hyper optimizer: -2.071104019770442
Params used for optimizer: [[1.70219931 0.0446147  2.30214515 0.         5.2373898 ]
 [1.31195504 1.34393467 0.68683291 0.56937038 0.24570413]],
and params used for hyper optimizer: [8.92903817 9.0915074  3.57538043]
